# Data&Data Internship Assignment

This is the notebook that you should complete.

Please ensure that your code is clean, modular, and easy to follow. Feel free to create your own modules and import them into the notebook if it helps with clarity and structure.

 Feel free to add or delete cells in the notebook depending on you needs.

In [ ]:
# Imports (import the libraries you need)


## Part 1: CRUD Operations

Either use SQLThunder or implement a class that supports the four basic CRUD operations: **Create, Read, Update, and Delete**.

SQLThunder docs: https://sqlthunder.readthedocs.io/en/latest/

In [ ]:
# Class implementation or SQLThunder DBClient initialization


Create a simple table for demonstration purposes, and perform the four operations using just one row (the content is up to you)

In [ ]:
# Create table


In [ ]:
# Create operation (i.e. insert)


In [ ]:
# Read operation


In [ ]:
# Update operation


In [ ]:
# Delete operation


## Part 2: Web Scraping

Scrape data from profumeriaideale.com website for Valmont.

**Note**: Selenium is cool ;)

In [ ]:
# Scraping


In [ ]:
# More scraping ?


In [ ]:
# Even more scraping ?


## Part 3: Cleaning and Inserting Data

Cleaning

In [ ]:
# Cleaning


In [ ]:
# More cleaning ?


Insertion using class/SQLThunder form Part 1

In [ ]:
# Insert data in your local db (you might need to create a new table)


## Part 4: Entity Matching

Try to match each scraped data to official ref if it exists.

In [ ]:
# Load data


In [ ]:
# Match


In [ ]:
# More Matching


In [ ]:
# Lots of Matching


In [ ]:
# Now you can add as manyyyy cells as you want to match


### Output Excel

Please save your result in an excel in the format mentioned in the README.MD and submit it to Kaggle. Don't forget to send the repo by email.